In [ ]:
!nvidia-smi

Sun Dec 12 15:47:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    63W / 149W |    222MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# https://drive.google.com/file/d/1hbhObewzttZQz6w8ggyfDMgecG4AbdZ9/view?usp=sharing
!gdown --id '1hbhObewzttZQz6w8ggyfDMgecG4AbdZ9'

Downloading...
From: https://drive.google.com/uc?id=1hbhObewzttZQz6w8ggyfDMgecG4AbdZ9
To: /content/df.csv
100% 2.55M/2.55M [00:00<00:00, 72.7MB/s]


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv('df.csv', index_col='id')
df.head()

,host_since,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,security_deposit,availability_30,availability_365,calculated_host_listings_count,room_type_Shared room,zipcode_1062,host_verifications_email,host_verifications_government_id,host_verifications_way_num,amenities_Bedlinens,amenities_Breakfast,amenities_Cleaningbeforecheckout,amenities_Family/kidfriendly,amenities_Gym,amenities_Heating,amenities_Kitchen,amenities_Suitableforevents,amenities_way_num,price
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015,3733.0,4.0,4.0,52.534537,13.402557,3,1.0,1.0,200.0,0,141,4,0,0,1,1,6,1,0,0,1,1,1,1,0,29,60.0
2695,3704.0,1.0,1.0,52.548513,13.404553,2,1.0,1.0,0.0,0,0,1,0,0,1,1,5,0,0,0,0,0,1,1,0,10,17.0
3176,3671.0,1.0,1.0,52.534996,13.417579,4,1.0,1.0,200.0,0,220,1,0,0,1,1,7,0,0,0,1,0,1,1,0,19,90.0
3309,3652.0,1.0,1.0,52.498855,13.349065,2,1.0,1.0,250.0,0,297,1,0,0,1,1,6,1,0,0,0,0,1,0,0,29,26.0
7071,3462.0,1.0,1.0,52.543157,13.415091,2,1.0,1.0,0.0,15,26,1,0,0,1,1,5,1,0,0,1,0,1,0,0,13,42.0


In [ ]:
y = df['price']
x = df.drop(['price'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

# 70% train, 10% valid, 10% test, 10% stacking
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.125, random_state=42)
x_test, x_stack, y_test, y_stack = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(x_train)
x_train_std = pd.DataFrame(scaler.transform(x_train), index=x_train.index, columns=x_train.columns)
x_valid_std = pd.DataFrame(scaler.transform(x_valid), index=x_valid.index, columns=x_valid.columns)
x_test_std = pd.DataFrame(scaler.transform(x_test), index=x_test.index, columns=x_test.columns)
x_stack_std = pd.DataFrame(scaler.transform(x_stack), index=x_stack.index, columns=x_stack.columns)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

result = []

for dense_num in range(1, 11):  # 幾層幾層(1~10)
  for batch_size_num in [128, 256, 512]:
    for epochs_num in [20, 50, 100, 200, 500]:
      cnn_list = [Dense(16) for i in range(dense_num)] + [Dense(1)]
      cnn = Sequential(cnn_list)
      cnn.compile(loss='mean_absolute_error', optimizer='adam')
      cnn.fit(x_train_std, y_train, epochs=epochs_num, batch_size=batch_size_num, verbose=0)
      y_train_pred = cnn.predict(x_train_std, verbose=0)
      y_valid_pred = cnn.predict(x_valid_std, verbose=0)
      result.append([dense_num, batch_size_num, epochs_num, mae(y_train, y_train_pred), mae(y_valid, y_valid_pred)])
      print('{} Layers, Batch Size = {}, epochs = {}: Train MAE = {:4f}, Test MAE = {:4f}'\
            .format(dense_num, batch_size_num, epochs_num, mae(y_train, y_train_pred), mae(y_valid, y_valid_pred)))

1 Layers, Batch Size = 128, epochs = 20: Train MAE = 30.033075, Test MAE = 28.389987
1 Layers, Batch Size = 128, epochs = 50: Train MAE = 30.028508, Test MAE = 28.316069
1 Layers, Batch Size = 128, epochs = 100: Train MAE = 30.030883, Test MAE = 28.341128
1 Layers, Batch Size = 128, epochs = 200: Train MAE = 30.031502, Test MAE = 28.388869
1 Layers, Batch Size = 128, epochs = 500: Train MAE = 30.030720, Test MAE = 28.293560
1 Layers, Batch Size = 256, epochs = 20: Train MAE = 30.934392, Test MAE = 29.534440
1 Layers, Batch Size = 256, epochs = 50: Train MAE = 30.027908, Test MAE = 28.311876
1 Layers, Batch Size = 256, epochs = 100: Train MAE = 30.030226, Test MAE = 28.211784
1 Layers, Batch Size = 256, epochs = 200: Train MAE = 30.028657, Test MAE = 28.270225
1 Layers, Batch Size = 256, epochs = 500: Train MAE = 30.028439, Test MAE = 28.284420
1 Layers, Batch Size = 512, epochs = 20: Train MAE = 51.575803, Test MAE = 50.314230
1 Layers, Batch Size = 512, epochs = 50: Train MAE = 30.072